### 36.1 The sqlite3 Module

We'll perform the following operations for employee data:

- Create database and table
- Insert multiple employee records
- Fetch all employees
- Fetch by condition (e.g., salary > X)
- Update employee data
- Delete an employee
- Count records
- Use parameterized queries for security
- Commit and close



In [3]:
import sqlite3

In [5]:
# Connect to the database
conn = sqlite3.connect('employee.db')
cursor = conn.cursor()

In [7]:
# Create a employees table
cursor.execute(''' 
    CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            department TEXT NOT NULL,
            salary REAL NOT NULL,
            join_date TEXT
    )
''')

In [9]:
# insert a single row into that table
cursor.execute('''
    INSERT INTO employees (name, department, salary, join_date) VALUES ("Alice", "HR", 60000, "2023-01-15")
''')

In [15]:
# Selecting the data from the table
cursor.execute('''
    SELECT * FROM employees
''')
cursor.fetchall()

[(1, 'Alice', 'HR', 60000.0, '2023-01-15')]

In [17]:
# Inserting more data rows
employee_data = [
    ("Bob", "Engineering", 85000, "2022-09-10"),
    ("Charlie", "Marketing", 55000, "2024-03-01"),
    ("David", "Engineering", 90000, "2021-11-20"),
]

# executing multiple inserts
cursor.executemany('''
    INSERT INTO employees (name, department, salary, join_date) VALUES (?, ?, ?, ?)
''', employee_data)


In [21]:
# Selecting the data from the table
cursor.execute('''
    SELECT * FROM employees
''')
cursor.fetchall()

[(1, 'Alice', 'HR', 60000.0, '2023-01-15'),
 (2, 'Bob', 'Engineering', 85000.0, '2022-09-10'),
 (3, 'Charlie', 'Marketing', 55000.0, '2024-03-01'),
 (4, 'David', 'Engineering', 90000.0, '2021-11-20')]

In [25]:
# Fetch employees with salary > 60000
print("Employees with salary > 60000:")
# SELECT name, salary FROM employees WHERE salary > 60000;
cursor.execute("SELECT name, salary FROM employees WHERE salary > ?", (60000,))
for row in cursor.fetchall():
    print(row)
print()

Employees with salary > 60000:
('Bob', 85000.0)
('David', 90000.0)



In [27]:
# Update salary of an employee
cursor.execute("UPDATE employees SET salary = ? WHERE name = ?", (95000, "David"))

In [29]:
# Selecting the data from the table
cursor.execute('''
    SELECT * FROM employees
''')
cursor.fetchall()

[(1, 'Alice', 'HR', 60000.0, '2023-01-15'),
 (2, 'Bob', 'Engineering', 85000.0, '2022-09-10'),
 (3, 'Charlie', 'Marketing', 55000.0, '2024-03-01'),
 (4, 'David', 'Engineering', 95000.0, '2021-11-20')]

In [31]:
# Delete an employee by name
cursor.execute("DELETE FROM employees WHERE name = ?", ("Charlie",))


In [33]:
# Selecting the data from the table
cursor.execute('''
    SELECT * FROM employees
''')
cursor.fetchall()

[(1, 'Alice', 'HR', 60000.0, '2023-01-15'),
 (2, 'Bob', 'Engineering', 85000.0, '2022-09-10'),
 (4, 'David', 'Engineering', 95000.0, '2021-11-20')]

In [37]:
# Count the number of employees
cursor.execute("SELECT COUNT(*) FROM employees")
cursor.fetchone()[0]

3

In [39]:
count = cursor.fetchone()[0] 
print(f"Total number of employees: {count}\n")

TypeError: 'NoneType' object is not subscriptable

The above gives an error because the cursor object is a generator

In [41]:
# Count the number of employees
cursor.execute("SELECT COUNT(*) FROM employees")
count = cursor.fetchone()[0] 
print(f"Total number of employees: {count}\n")

Total number of employees: 3



In [43]:
# Close the connection
conn.commit()

In [45]:
conn.close()

##### Re-open the database

In [49]:
# Connect to the database
conn = sqlite3.connect('employee.db')
cursor = conn.cursor()

In [51]:
# Selecting the data from the table
cursor.execute('''
    SELECT * FROM employees
''')
cursor.fetchall()

[(1, 'Alice', 'HR', 60000.0, '2023-01-15'),
 (2, 'Bob', 'Engineering', 85000.0, '2022-09-10'),
 (4, 'David', 'Engineering', 95000.0, '2021-11-20')]

##### You can see the data is still available

### 36.2 The pickle Module

Reference: https://docs.python.org/3/library/pickle.html#module-pickle

You are building a Python application that needs to save user session data to disk and retrieve it later using the pickle module.

Requirements:

Define a UserSession class with the following attributes:

    username (string)
    last_login_time (datetime)
    preferences (dictionary of user settings, like theme, language, etc.)

- Create at least two instances of the UserSession class with dummy data.
- Serialize (save) the list of session objects to a file called sessions.pkl.
- Later, deserialize (load) the list of sessions from the file.
- Print out each user’s data in a readable format.

Deliverables:

- Python code demonstrating creation, pickling, and unpickling.
- Output showing the loaded data from the pickle file.
- Use of proper error handling (e.g., FileNotFoundError or EOFError).

Challenge:

Add a feature that updates a user’s preferences and re-pickles the modified session list.



In [55]:
import pickle
from datetime import datetime 

##### Define the class and create objects

In [65]:
# Define the user session class
class UserSession:

    def __init__(self, username, last_login_time, preferences):
        self.username = username
        self.last_login_time = last_login_time
        self.preferences = preferences

    def __repr__(self):
        return( f"UserSession(username='{self.username}', last_login_time={self.last_login_time}, self.preferences={self.preferences})")

In [67]:
s1 = UserSession('Anil', datetime.now(), {'theme': "dark", "language":"English"})
s2 = UserSession('Sunil', datetime.now(), {'theme': "light", "language":"French"})

In [69]:
s1

UserSession(username='Anil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'dark', 'language': 'English'})

In [71]:
s2

UserSession(username='Sunil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'light', 'language': 'French'})

##### How to store the objects in a file

In [75]:
sessions = [s1, s2]

In [77]:
# Serialize the session list
with open("sessions.pkl", "wb") as f:
    pickle.dump(sessions, f)

##### How to re-store the objects from a pickle file

In [80]:
try:
    with open("sessions.pkl", "rb") as f:
        loaded_sessions = pickle.load(f)
    for session in loaded_sessions:
        print(session)
except Exception as e:
    print(f"Exception {e}")

UserSession(username='Anil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'dark', 'language': 'English'})
UserSession(username='Sunil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'light', 'language': 'French'})


In [82]:
# Just to make sure they are objects that are retrieved
loaded_sessions[0].preferences["theme"] = "light"
loaded_sessions[0].preferences["language"] = "Spanish"

In [84]:
for session in loaded_sessions:
    print(session)

UserSession(username='Anil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'light', 'language': 'Spanish'})
UserSession(username='Sunil', last_login_time=2025-04-09 13:15:49.988548, self.preferences={'theme': 'light', 'language': 'French'})


In [88]:
# Re-Serialize the session list
with open("sessions.pkl", "wb") as f:
    pickle.dump(sessions, f)

##### The sessions.pkl will have the updated session information